In [1]:
# importing necessary library
import pymongo as py
import pandas as pd

In [38]:
# connecting cluster to the client(python texteditor)
connection_uri = input('Enter the mongodb connection uri : ')
client = py.MongoClient(connection_uri)

In [37]:
# listing database names
client.list_database_names()

### Importing documents into database

In [4]:
# reading data from json using pandas
df = pd.read_json('~/Desktop/students.json',lines=True)
df

,_id,name,scores
0,0,aimee Zank,"[{'score': 1.463179736705023, 'type': 'exam'},..."
1,1,Aurelia Menendez,"[{'score': 60.06045071030959, 'type': 'exam'},..."
2,2,Corliss Zuk,"[{'score': 67.03077096065002, 'type': 'exam'},..."
3,3,Bao Ziglar,"[{'score': 71.64343899778332, 'type': 'exam'},..."
4,4,Zachary Langlais,"[{'score': 78.68385091304332, 'type': 'exam'},..."
...,...,...,...
195,195,Linnie Weigel,"[{'score': 52.44578368517977, 'type': 'exam'},..."
196,196,Santiago Dollins,"[{'score': 52.04052571137036, 'type': 'exam'},..."
197,197,Tonisha Games,"[{'score': 38.51269589995049, 'type': 'exam'},..."
198,198,Timothy Harrod,"[{'score': 11.9075674046519, 'type': 'exam'}, ..."


In [5]:
# changing dataframe to dictionary format
data = df.to_dict(orient = 'records')

In [6]:
# creating a database and a collection
db = client['students']
collection = db['details']

In [7]:
# inserting record into the details collection
collection.insert_many(data)

In [36]:
list(collection.find())

# 1. Students who scored maximum scores in all (exam,quiz and homework)

In [9]:
# max_marks_agg id a cursor to the documents containing max marks for each type of exam
max_marks_per_type = collection.aggregate([
                            {'$unwind' : '$scores'}, # deconstructing a array field and creating documents with each array element
                            {'$group' : {'_id': '$scores.type','max_marks':{'$max':'$scores.score'}}}, #group based of the type of exam and finding max marks in it
                        ])
# max_marks_per_type -output = exam type and max marks only

# looping through each document matching max_marks for its type
for i in max_marks_per_type:
    x = collection.aggregate([
        {'$unwind':'$scores'}, # decontructing a array field -scores
        {'$match' : {'$and' : [{'scores.type': i['_id']},{'scores.score' : i['max_marks']}]}}]) # querying documents which match the marks and the type from the above result

    for j in x: # printing the students list from each type who has scored the highest 
        print(f"In {j['scores']['type']} , {j['name']} (id : {j['_id']}) has scored the highest mark {j['scores']['score']}")

In quiz , Cody Strouth (id : 69) has scored the highest mark 99.80348240553108
In exam , Margart Vitello (id : 136) has scored the highest mark 99.33685767140612
In homework , Whitley Fears (id : 178) has scored the highest mark 99.77237745070993


# 2. Students who scored below average in the exam and passed?

In [10]:
# getting average marks of the exam
exam_avg = collection.aggregate([                       
                    {'$unwind' : '$scores'},
                    {'$match' : {'scores.type':'exam'}},
                    {'$group' : {'_id':'$scores.type','avg_mark':{'$avg' : '$scores.score'}}}
])
exam_avg = list(exam_avg)[0]['avg_mark']

# getting list of students having marks below avg and passed in the exam
std_bavg_apass_exam1 = collection.aggregate([
                    {'$unwind' : '$scores'},
                    {'$match' :{'$and' : [{'scores.type':'exam'},{'scores.score' :{'$gte': 40,'$lt': exam_avg}}]}},
                    {'$project' : {'name':1}}
])
std_bavg_apass_exam = list(std_bavg_apass_exam1)

In [11]:
print(f'No. of students who passed and scored below average in exam : {len(std_bavg_apass_exam)}')

No. of students who passed and scored below average in exam : 23


In [12]:
for i in std_bavg_apass_exam:
    print(i)

{'_id': 5, 'name': 'Wilburn Spiess'}
{'_id': 10, 'name': 'Denisha Cast'}
{'_id': 19, 'name': 'Gisela Levin'}
{'_id': 20, 'name': 'Tressa Schwing'}
{'_id': 21, 'name': 'Rosana Vales'}
{'_id': 23, 'name': 'Tamika Schildgen'}
{'_id': 29, 'name': 'Gwyneth Garling'}
{'_id': 42, 'name': 'Kayce Kenyon'}
{'_id': 45, 'name': 'Terica Brugger'}
{'_id': 57, 'name': 'Chad Rahe'}
{'_id': 68, 'name': 'Jenise Mcguffie'}
{'_id': 89, 'name': 'Cassi Heal'}
{'_id': 100, 'name': 'Demarcus Audette'}
{'_id': 109, 'name': 'Flora Duell'}
{'_id': 126, 'name': 'Quincy Danaher'}
{'_id': 128, 'name': 'Alix Sherrill'}
{'_id': 142, 'name': 'Laureen Salomone'}
{'_id': 143, 'name': 'Gwyneth Garling'}
{'_id': 155, 'name': 'Aleida Elsass'}
{'_id': 164, 'name': 'Alica Pasley'}
{'_id': 167, 'name': 'Malisa Jeanes'}
{'_id': 184, 'name': 'Harriett Velarde'}
{'_id': 193, 'name': 'Mariela Sherer'}


# 3. students who scored below pass mark and assigning them as fail, and above pass mark as pass in all the categories.


1. students who scored below pass mark in each categories
2. students who scored above pass mark in each categories
3. students who are assigned fail (if one or more categories failed)
4. students who are assigned pass (if all the categories are above pass mark)

### 3.1 students who scored below pass mark in each categories


In [13]:
# getting student ids who Failed in each categories


std_b_pass_exam1 = collection.aggregate([
                        {'$unwind' : '$scores'},
                        {'$match' : {'$and' : [{'scores.type':'exam'},{'scores.score' :{'$lt': 40}}]}},
                        {'$project' :{'_id' : 1}}
])
std_b_pass_quiz1 = collection.aggregate([
                        {'$unwind' : '$scores'},
                        {'$match' : {'$and' : [{'scores.type':'quiz'},{'scores.score' :{'$lt': 40}}]}},
                        {'$project' :{'_id' : 1}}
])
std_b_pass_homework1 = collection.aggregate([
                        {'$unwind' : '$scores'},
                        {'$match' : {'$and' : [{'scores.type':'homework'},{'scores.score' :{'$lt': 40}}]}},
                        {'$project' :{'_id' : 1}}
])

# creating a array  of embedded documents

# below mongodb query creates a array named Results and inserts documents {result_name : Fail}
# students who scored below pass in exam 
for i in std_b_pass_exam1:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Results' : {'result_exam':'Fail'}}})
# students who scored below pass in quiz
for i in std_b_pass_quiz1:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Results' : {'result_quiz':'Fail'}}})
# students who scored below pass in homework
for i in std_b_pass_homework1:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Results' : {'result_homework':'Fail'}}})


### 3.2. students who scored above pass mark in each categories

In [14]:
# getting student names and id who passed in each categories

std_a_pass_exam = collection.aggregate([
                    {'$unwind' : '$scores'},
                    {'$match' :{'$and' : [{'scores.type':'exam'},{'scores.score' :{'$gte': 40}}]}},
                    {'$project' : {'name':1}}])
std_a_pass_quiz = collection.aggregate([
                        {'$unwind' : '$scores'},
                        {'$match' : {'$and' : [{'scores.type':'quiz'},{'scores.score' :{'$gte': 40}}]}},
                        {'$project' :{'name' : 1}}
                    ])
std_a_pass_homework = collection.aggregate([
                        {'$unwind' : '$scores'},
                        {'$match' : {'$and' : [{'scores.type':'homework'},{'scores.score' :{'$gte': 40}}]}},
                        {'$project' :{'name' : 1}}
])

# creating a array  of embedded documents

# below mongodb query creates/finds an array named Results and inserts documents {result_name : pass}
# students who scored above pass in exam
for i in std_a_pass_exam:
    collection.update_one({'_id':i['_id']},
                     {'$push': {'Results' : {'result_exam':'Pass'}}})
# students who scored above pass in quiz
for i in std_a_pass_quiz:
    collection.update_one({'_id':i['_id']},
                     {'$push': {'Results':{'result_quiz':'Pass'}}})
# students who scored above pass in homework
for i in std_a_pass_homework:
    collection.update_one({'_id':i['_id']},
                     {'$push': {'Results':{'result_homework':'Pass'}}})

### 3.3. students who are assigned fail (if one or many of the categories failed)


In [15]:
# getting students who failed in atleast one category
std_failed1 = collection.aggregate([
                    {'$unwind': '$scores'},
                    {'$match' :{'scores.score' :{'$lt': 40}}},
                    {'$group' : {'_id': '$_id','count' : {'$sum':1}}},
])

# inserting a field called overall result and assigning them as fail 

for i in std_failed1:
    collection.update_one(
                     {'_id': i['_id']},
                     {'$set': {'overall_result': 'Fail'}}
    )

### 3.4. students who are assigned pass (if all the categories are above pass mark)


In [16]:
# getting students who passed in all category
std_passed1 = collection.aggregate([
                    {'$unwind': '$scores'},
                    {'$match' :{'scores.score' :{'$gte': 40}}},
                    {'$group' : {'_id': '$_id','count' : {'$sum':1}}}
])

# inserting a field called overall result and assigning them as pass

for i in std_passed1:
    if i['count'] == 3:
        collection.update_one(
            {'_id' : i['_id']},
            {'$set': {'overall_result':'Pass'}}
    )

# 4. The total and average of the exam, quiz and homework and store them in a separate collection

1. finding the total and average of the exam ,quiz,homework for each students and storing them in a new collection
2. finding the total and average of the exam ,quiz,homework and storing them in a new collection


### 4.1. finding the total and average of the exam ,quiz,homework for each students and storing them in a new collection


In [35]:
# finding the total and average of the exam , quiz ,homework for each students


std_avg_total = collection.aggregate([
                {'$unwind': '$scores'},
                {'$group' :
                     {'_id' : '$_id', 
                     'avg': {'$avg' : '$scores.score'},
                     'total' : {'$sum':'$scores.score'}}},
                {'$project':{'_id':1,'name':1,'avg':1,'total':1}},
                {'$sort': {'_id':1}},
])

# list of ids and their totals and average of 3 categories
std_list_avg_total = list(std_avg_total)
# std_list_avg_total

In [18]:
# creating a collection named 'students_avg_total' and storing the above result in it
db['students_avg_total'].insert_many(std_list_avg_total)

### 4.2. finding the total and average of the exam ,quiz,homework and storing them in a new collection


In [19]:
# find the total and average of the exam , quiz and home work
type_avg_total = collection.aggregate([
                    {'$unwind': '$scores'},
                    {'$group': 
                             {'_id' : '$scores.type',
                             'total':{'$sum':'$scores.score'},
                             'avg' :{'$avg':'$scores.score'}}
                    },
                    {'$sort':{'_id':1}}
])
list_type_avg_total = list(type_avg_total)

In [20]:
# creating a collection named 'type_avg_total' and storing the above result in it
db['type_avg_total'].insert_many(list_type_avg_total)

# 5.  Create a new collection which consists of students who scored below average and above 40% in all the categories.


1. finding students who scored below average and above 40 % in all the categories
2. finding students who scored below average and above 40% for each categories

In [21]:
# getting avg for each categories
avg_exam = list_type_avg_total[0]['avg']
avg_quiz = list_type_avg_total[2]['avg']
avg_homework = list_type_avg_total[1]['avg']
print(f'avg_quiz : {avg_quiz}\navg_homework : {avg_homework}\navg_exam : {avg_exam}')

avg_quiz : 48.99672319430254
avg_homework : 67.81869620661149
avg_exam : 48.67367075950175


### 5.1 finding students who scored below average and above 40 % in all the categories


In [22]:
bavg_apass_all1 = collection.aggregate([
                            {'$unwind': '$scores'},
                            {'$match': 
                             {'$and': [
                                   {'$and': [{'scores.type':'exam'},
                                             {'scores.score':{'$lt':avg_exam,'$gte' :40}}]},
                                    
                                   {'$and': [{'scores.type':'quiz'},
                                             {'scores.score':{'$lt':avg_quiz,'$gte' :40}}]},
                                    
                                   {'$and': [{'scores.type':'homework'},
                                             {'scores.score':{'$lt':avg_homework,'$gte' :40}}]}
                                ]}
                                },
                        {'$group': {'_id':'$_id','count':{'$sum':1}}}
])
bavg_apass_all = list(bavg_apass_all1)
# print(len(bavg_apass_all))
# since there is no one matchs the condition for all the categories the data is zero and can't create a collection

### 5.2 finding students who scored below average and above 40% for each categories

In [23]:
# list of students who scored below average and above 40 % for each categories
bavg_apass_any1 = collection.aggregate([
                            {'$unwind': '$scores'},
                            {'$match': 
                             {'$or': [
                                   {'$and': [{'scores.type':'exam'},
                                             {'scores.score':{'$lt':avg_exam,'$gte' :40}}]},
                                    
                                   {'$and': [{'scores.type':'quiz'},
                                             {'scores.score':{'$lt':avg_quiz,'$gte' :40}}]},
                                    
                                   {'$and': [{'scores.type':'homework'},
                                             {'scores.score':{'$lt':avg_homework,'$gte' :40}}]}
                                ]}
                                },
])
bavg_apass_any = list(bavg_apass_any1)
# print(len(bavg_apass_any))


In [24]:
# initializing a collection name
std_bavg_apass = db['std_bavg_apass']
# this collection consists of documents containg id , name , categories in which the student scored below average and above 40% in each categories(array of embedded documents)

In [25]:
# ids of the students who scored below average and above 40% for each categories
bavg_apass_any_ids = collection.aggregate([
                            {'$unwind': '$scores'},
                            {'$match': 
                             {'$or': [
                                   {'$and': [{'scores.type':'exam'},
                                             {'scores.score':{'$lt':avg_exam,'$gte' :40}}]},
                                    
                                   {'$and': [{'scores.type':'quiz'},
                                             {'scores.score':{'$lt':avg_quiz,'$gte' :40}}]},
                                    
                                   {'$and': [{'scores.type':'homework'},
                                             {'scores.score':{'$lt':avg_homework,'$gte' :40}}]}
                                ]}
                                },
                        {'$group': {'_id':'$_id'}}
])
ids_bavg_apass = list(bavg_apass_any_ids)

In [26]:
# creating a collection using ids
std_bavg_apass.insert_many(ids_bavg_apass)

In [27]:
# updating the documents of the collection 'std_bavg_apass'
for i in bavg_apass_any:
    name = i['name']
    type_1 = i['scores']['type']
    std_bavg_apass.update_one({'_id':i['_id']},
                           {'$set': {'name':name}})
    std_bavg_apass.update_one({'_id':i['_id']},
                           {'$push': {'apass_bavg':{'type':type_1}}})

# 6. Create a new collection which consists of students who scored below the fail mark in all the categories.

1. creating a collection in which students failed in any categories
2. creating a collection in which students passed in all

### 6.1. creating a collection in which students failed in any categories


In [28]:
# list of students failed in atleast one category
std_failed1 = collection.aggregate([
                    {'$unwind': '$scores'},
                    {'$match' :{'scores.score' :{'$lt': 40}}},
])


In [29]:
# initializing a collection
std_fail_any = db['std_fail_any']

In [30]:
# getting the ids of the students failed
std_failed_ids = collection.aggregate([
                            {'$unwind': '$scores'},
                            {'$match' :{'scores.score' :{'$lt': 40}}},
                            {'$group': {'_id':'$_id'}}
])
ids_std_failed = list(std_failed_ids)
# creating a collection named 'std_fail_any' using only ids of the students failed
std_fail_any.insert_many(ids_std_failed)

In [31]:
# updating the documents of the collection 'std_fail_any' with the name and 
# failed types(array of embedded documents)
for i in std_failed1:
    name = i['name']
    type_1 = i['scores']['type']
    std_fail_any.update_one({'_id':i['_id']},
                           {'$set': {'name':name}})
    std_fail_any.update_one({'_id':i['_id']},
                           {'$push': {'Failed type(s)':{'type':type_1}}})

### 6.2. creating a collection in which students passed in all

In [32]:
# ids of the students who passed in each categories
std_pass_any_ids = collection.aggregate([
                    {'$unwind': '$scores'},
                    {'$match' :{'scores.score':{'$gte': 40}}},
                    {'$group' :{'_id':{'_id':'$_id','name':'$name'},'count':{'$sum':1}}},
])
ids_std_pass_list = list(std_pass_any_ids)

In [33]:
# initializing a collection
std_allpass = db['std_allpass']
# this collection consists of id and name of the students who passed all the categories

In [34]:
# filtering the ids of the students who passed in all categories and uploading their information into a collection named 'std_allpass'
for i in ids_std_pass_list:
   if i['count'] == 3:
        firstpair = i['_id']
        id1 = firstpair['_id']
        name = firstpair['name']
        std_allpass.insert_one({'_id':id1,'name':name})